In [1]:
import numpy as np
import os
os.getcwd()

data = np.load("../data/processed/supervised_train.npz")
X, y = data["X"], data["y"]

print("X shape:", X.shape)          # Expect (N, 12, 8, 8)
print("y shape:", y.shape)          # Expect (N,)
print("dtype:", X.dtype, y.dtype)
print("action_space:", y.max() + 1, "max index:", y.max())

X shape: (348113, 12, 8, 8)
y shape: (348113,)
dtype: float32 int64
action_space: 4661 max index: 4660
